This notebook addresses a parallel machine scheduling problem. The objective studied is minimizing the sum of weighted tardiness. 
Each job needs to be processed by one of the batching machines. Batching machines can process simultaneously up to a specified number of the jobs of a particular family (type).  

Each job is associated with:

- proccesing time
- weight
- family (type)
- due date

The scheduling problem can be represented by P| <em>batch, fmls</em> | $ \sum\limits_{j=1}^{n}w_𝑗T_j $  . This problem is known to be NP-hard. IBM ILOG CPLEX Optimization Studio includes solvers for both Mathematical and Constraint Programming. Constraint Programming is particularly efficient and useful to tackle detailed scheduling problems. By using docplex.cp python package, you can easily formulate and solve scheduling problems in python notebooks. Below is an example formulation with randomly generated sample data to provide a better understanding of the problem and the model.

In [1]:
nbrOfJobs = 10
jobs= [*range(0, nbrOfJobs)] 

In [2]:
nbrOfMachines = 3
machines= [*range(0, nbrOfMachines)] 

In [3]:
#generate a job type and processing time for each job
import random
nbrOfJobTypes = 3   #job families
types = [*range(0, nbrOfJobTypes)] 
minProcessingTime=10
maxProcessingTime=40
jobTypeProcessingTimes = [random.randint(minProcessingTime, maxProcessingTime) for t in types]
jobTypes = [random.randint(0, nbrOfJobTypes-1) for j in jobs]
processingTimes = [jobTypeProcessingTimes[jobTypes[j]] for j in jobs]

In [4]:
minWeight=1
maxWeight=5
weights= [random.randint(minWeight, maxWeight) for j in jobs]

In [5]:
minDueDate=10
maxDueDate=60
dueDates= random.sample(range(minDueDate, maxDueDate), len(jobs))

In [6]:
import pandas as pd
JobsTable = pd.DataFrame(columns=['job', 'type','processing_time', 'weight', 'due date'])
for j in jobs:
    JobsTable=JobsTable.append({'job': j,'type': jobTypes[j],'processing_time':processingTimes[j],'weight':weights[j],'due date':dueDates[j]}, ignore_index=True)
print(JobsTable)

  job type processing_time weight due date
0   0    0              14      4       34
1   1    1              12      3       18
2   2    0              14      1       11
3   3    2              28      5       35
4   4    2              28      4       15
5   5    1              12      2       49
6   6    2              28      1       33
7   7    1              12      2       30
8   8    0              14      4       48
9   9    2              28      3       21


In [7]:
#define capacity of batching machine
batchingCapacity=2

In [8]:
from docplex.cp.model import *
mdl = CpoModel(name='parallelMachineScheduling_TWT_Batching') 

In [9]:
# define production processing interval of each job at each machine
processing_itv_vars = [[mdl.interval_var(optional=True, size=processingTimes[j], name="interval_job{}_machine{}".format(j,m)) for m in machines] for j in jobs] 

In [10]:
s=[m for m in machines]
for m in machines:
    s[m] = mdl.state_function()

In [11]:
jobs_atAtime=[m for m in machines]
for m in machines:
    jobs_atAtime[m] = sum([pulse(processing_itv_vars[j][m],1) for j in jobs])  #to be used in batching constraints 

In [12]:
#minimize total weighted tardiness time 
objective = mdl.sum(mdl.max([0,(mdl.end_of(processing_itv_vars[j][m])-dueDates[j])])*weights[j] for j in jobs for m in machines)
mdl.add(mdl.minimize(objective)) 

In [13]:
#each job should be assigned to a machine
for j in jobs:
    mdl.add(mdl.sum([mdl.presence_of(processing_itv_vars[j][m]) for m in machines]) == 1)

In [14]:
#only jobs of the same family(type) can be processed simultaneously by the same machine
for m in machines:
    for j in jobs:
        mdl.add(mdl.always_equal(s[m], processing_itv_vars[j][m], jobTypes[j]))

In [15]:
#capacity cannot be exceeded
for m in machines:
    mdl.add(jobs_atAtime[m] <= batchingCapacity) 

In [16]:
msol= mdl.solve(log_output=True)

In [17]:
print("Solution: ")
msol.print_solution()

Solution: 
-------------------------------------------------------------------------------
Model constraints: 43, variables: integer: 0, interval: 30, sequence: 0
Solve status: Optimal, Fail status: SearchHasFailedNormally
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 5.51 sec
-------------------------------------------------------------------------------
Objective values: (88,), bounds: (88,), gaps: (0,)
interval_job0_machine0: absent
interval_job0_machine1: absent
interval_job0_machine2: (start=12, end=26, size=14, length=14)
interval_job1_machine0: absent
interval_job1_machine1: absent
interval_job1_machine2: (start=0, end=12, size=12, length=12)
interval_job2_machine0: absent
interval_job2_machine1: absent
interval_job2_machine2: (start=12, end=26, size=14, length=14)
interval_job3_machine0: absent
interval_job3_machine1: (start=0, end=28, size=28, length=28)
interval_job3_machine2: absent
interval_job4_machine0: (start=0, end=28, size=28, length=2